In [1]:
from keras.layers import Input, ZeroPadding3D, Conv3D, BatchNormalization, Activation, SpatialDropout3D, MaxPooling3D, \
    TimeDistributed, Flatten, Bidirectional, GRU, Dense, AveragePooling3D
from keras import Model
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        
import keras
import numpy as np
from data_gen_stanford import DataGenerator

Using TensorFlow backend.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
def get_Lipnet(n_classes=51, summary=False):
    input_layer = Input(name='the_input', shape=(75, 50, 100, 3), dtype='float32')
    x = Conv3D(32, (3, 5, 5), strides=(1, 2, 2), padding="same", kernel_initializer='he_normal', name='conv1')(input_layer)
    x = BatchNormalization(name='batc1')(x)
    x = Activation('relu', name='actv1')(x)
    x = SpatialDropout3D(0.5)(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max1')(x)

    x = Conv3D(64, (3, 5, 5), strides=(1, 1, 1), padding="same", kernel_initializer='he_normal', name='conv2')(x)
    x = BatchNormalization(name='batc2')(x)
    x = Activation('relu', name='actv2')(x)
    x = SpatialDropout3D(0.5)(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max2')(x)

    x = Conv3D(96, (3, 3, 3), strides=(1, 1, 1), padding="same", kernel_initializer='he_normal', name='conv3')(x)
    x = BatchNormalization(name='batc3')(x)
    x = Activation('relu', name='actv3')(x)
    x = SpatialDropout3D(0.5)(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max3')(x)
    
    x = TimeDistributed(Flatten())(x)

    x = Bidirectional(GRU(128, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'),
                            merge_mode='concat')(x)
    x = Bidirectional(GRU(128, return_sequences=True, kernel_initializer='Orthogonal', name='gru2'),
                            merge_mode='concat')(x)
    x = Flatten()(x)
    outputs = Dense(n_classes, kernel_initializer='he_normal', name='dense1', activation="sigmoid")(x)

    model = Model(inputs=input_layer, outputs=outputs)
    if summary:
        keras.utils.plot_model(model, 'network.png', show_shapes=True)
        print(model.summary())

    model.compile(optimizer=keras.optimizers.Adam(beta_1=0.9, beta_2=0.999, lr=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'mse', tf.keras.metrics.AUC()])
    return model

In [ ]:
if __name__ == "__main__":
    model = get_Lipnet(n_classes=51, summary=False)
    datagen = DataGenerator(batch_size=10, val_split=0.99)
    model.fit_generator(generator=datagen, epochs=1, shuffle=True, validation_data=datagen.get_valid_data())
        